# Section 506.2 - Allowable area determination
## Building Code
The allowable area of a building shall be determined in accordance with the applicable provisions of Sections 506.2.1 through 506.2.4 and Section 506.3. 

### TABLE 506.2a, b ALLOWABLE AREA FACTOR (At = NS, S1, S13R, or SM, as applicable) IN SQUARE FEET 

In [1]:
import sys
import os

# Add the path to the Rule Notebooks folder to the system path
sys.path.append(os.path.abspath('../..'))

# Import the find_value function from table_lookup.py
from ruleFunctions import find_value, check_value, get_building_info, create_scrollable_table

In [2]:
import json 

# Load the table from the JSON file 
with open('table_506_2.json', 'r') as f:
    table_506_2 = json.load(f)

In [3]:
df = create_scrollable_table(table_506_2, limit_column_name="Area Limit", max_height=400)

Occupancy Group,Sprinkler System,Construction Type,Subtype,Area Limit
A-1,NS,Type I,A,15500
A-1,NS,Type I,B,8500
A-1,NS,Type II,A,14000
A-1,NS,Type II,B,8500
A-1,NS,Type III,A,15000
A-1,NS,Type III,B,11500
A-1,NS,Type IV,HT,5500
A-1,NS,Type V,A,5500
A-1,S1,Type I,A,62000
A-1,S1,Type I,B,34000


**Note:** UL = Unlimited; NP = Not permitted;  
For SI: 1 square foot = 0.0929 m².  

- **NS** = Buildings not equipped throughout with an automatic sprinkler system  
- **S1** = Buildings a maximum of one story above grade plane equipped throughout with an automatic sprinkler system installed in accordance with Section 903.3.1.1  
- **SM** = Buildings two or more stories above grade plane equipped throughout with an automatic sprinkler system installed in accordance with Section 903.3.1.1  
- **S13R** = Buildings equipped throughout with an automatic sprinkler system installed in accordance with Section 903.3.1.2  

**Footnotes:**

a. See Chapters 4 and 5 for specific exceptions to the allowable height in this chapter.  
b. See Section 903.2 for the minimum thresholds for protection by an automatic sprinkler system for specific occupancies.  
c. New Group H occupancies are required to be protected by an automatic sprinkler system in accordance with Section 903.2.5.  
d. The NS value is only for use in evaluation of existing building area in accordance with the International Existing Building Code.  
e. New Group I-1 and I-3 occupancies are required to be protected by an automatic sprinkler system in accordance with Section 903.2.6. For new Group I-1 occupancies, Condition 1, see Exception 1 of Section 903.2.6.  
f. New and existing Group I-2 occupancies are required to be protected by an automatic sprinkler system in accordance with Section 903.2.6 and Section 1103.5 of the International Fire Code.  
g. New Group I-4 occupancies see Exceptions 2 and 3 of Section 903.2.6.  
h. New Group R occupancies are required to be protected by an automatic sprinkler system in accordance with Section 903.2.8.

In [4]:
from SPARQLWrapper import SPARQLWrapper, JSON

with open("../../../fuseki_endpoints.txt", "r") as file:
    endpoint_url = file.read()

sparql = SPARQLWrapper(endpoint_url)

In [5]:
# Call the function to get the building information
main_construction_type, subtype, occupancy_group, sprinkler_system, sprinkler_type, storey_num = get_building_info(endpoint_url)


#print(main_construction_type, subtype, occupancy_group, sprinkler_system, sprinkler_type, storey_num)

In [6]:
if sprinkler_system == "S":
    if int(storey_num) > 1: 
        sprinkler_system = "SM"
    else:
        sprinkler_system = "S1"

allowed_building_height = find_value(table_506_2, occupancy_group, sprinkler_system, main_construction_type, subtype)
print(allowed_building_height)

21000


In [7]:
with open("..\..\..\SPARQL\\uniqueOccupancy.sparql", "r") as file:
    sparql_query = file.read()

# Set the SPARQL query and the return format
sparql.setQuery(sparql_query)
sparql.setReturnFormat(JSON)

# Execute the query and get the results in JSON format
results = sparql.query().convert()

# Print the initial section message
print(f"This section is considered: Passed"
      f"\n\nThe allowable area of the building was determined in accordance with Section 506.2.4 as there is more than one occupancy in the multi-story building: \n")

# Process and format the results
story_dict = {}

for result in results["results"]["bindings"]:
    level_value = result["levelValue"]["value"]
    occupancies = result["uniqueOccupancies"]["value"].split(", ")  # Adjusted to the actual key name

    if level_value not in story_dict:
        story_dict[level_value] = []

    story_dict[level_value].extend(occupancies)

# Track the number of unique occupancies
num_unique_occupancies = 0

# Print the results in the desired format
for level, occupancies in sorted(story_dict.items()):
    unique_occupancies = sorted(set(occupancies))  # Remove duplicates and sort for better display
    if num_unique_occupancies < len(unique_occupancies):
        num_unique_occupancies = len(unique_occupancies)  # Count the number of unique occupancies
    print(f"• Story: {level}")
    for occupancy in unique_occupancies:
        print(f"  ◾ Occupancy: {occupancy}")
    print()  # Add a blank line after each story

This section is considered: Passed

The allowable area of the building was determined in accordance with Section 506.2.4 as there is more than one occupancy in the multi-story building: 

• Story: 01
  ◾ Occupancy: A-2
  ◾ Occupancy: A-3
  ◾ Occupancy: B
  ◾ Occupancy: R-1

• Story: 02
  ◾ Occupancy: R-1

• Story: 03
  ◾ Occupancy: R-1

• Story: 04
  ◾ Occupancy: R-1

